In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_dir = '/content/drive/MyDrive/glas'


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os
from sklearn.metrics import f1_score, jaccard_score
import numpy as np

In [ ]:
!pip install albumentations


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
class GLASDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.imgs = sorted(os.listdir(img_dir))
        self.masks = sorted(os.listdir(mask_dir))

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.imgs[idx])
        mask_path = os.path.join(self.mask_dir, self.masks[idx])
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"))

        # Pass named arguments to albumentations transformation
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        mask = torch.unsqueeze(mask, dim=0)
        mask = mask.type(torch.FloatTensor)
        return image, mask


In [ ]:
transform = A.Compose([
    A.Resize(256, 256),  # Resize both images and masks
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [ ]:
train_loader = DataLoader(GLASDataset(img_dir=f"{dataset_dir}/images",
                                      mask_dir=f"{dataset_dir}/masks/0",
                                      transform=transform),
                          batch_size=8, shuffle=True)

test_loader = DataLoader(GLASDataset(img_dir=f"{dataset_dir}/images",
                                     mask_dir=f"{dataset_dir}/masks/0",
                                     transform=transform),
                         batch_size=1, shuffle=False)

In [ ]:
class UNet(nn.Module):
    def __init__(self, pretrained=True):
        super(UNet, self).__init__()
        self.encoder = models.resnet18(pretrained=pretrained)
        self.encoder = nn.Sequential(*list(self.encoder.children())[:-2])  # Remove the FC layer

        self.upconv1 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.upconv3 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)

        # Upsample the final output to 256x256
        self.upsample = nn.Upsample(scale_factor=4, mode='bilinear', align_corners=True)

        self.final_conv = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)  # Encode the input
        x = self.upconv1(x)  # First upconv layer
        x = self.upconv2(x)  # Second upconv layer
        x = self.upconv3(x)  # Third upconv layer

        x = self.final_conv(x)  # Final convolution layer
        x = self.upsample(x)    # Upsample to 256x256
        return x


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, masks in dataloader:
            inputs = inputs.to(device)
            masks = masks.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}')

train_model(model, train_loader, criterion, optimizer)


Epoch 1/10, Loss: -0.21309308849629902
Epoch 2/10, Loss: -16.32132113547552
Epoch 3/10, Loss: -106.28546932765416
Epoch 4/10, Loss: -409.6653834751674
Epoch 5/10, Loss: -1106.127476283482
Epoch 6/10, Loss: -2756.521036783854
Epoch 7/10, Loss: -5636.5395740327385
Epoch 8/10, Loss: -10059.358735584077
Epoch 9/10, Loss: -17609.375395275296
Epoch 10/10, Loss: -27791.765671502977
